In [12]:
import pandas as pd

In [13]:
gpus_dataset = pd.read_csv("gpus.csv")

In [14]:
gpus_dataset.head()

,name,type,tdp_watts,TFLOPS32,TFLOPS16,GFLOPS32/W,GFLOPS16/W,memory,source
0,A100 PCIe 40/80GB,gpu,250,NaN,312.00,NaN,NaN,40.0,https://www.nvidia.com/content/dam/en-zz/Solut...
1,A100 SXM4 80 GB,gpu,400,NaN,312.00,NaN,NaN,80.0,https://www.nvidia.com/content/dam/en-zz/Solut...
2,AGX Xavier,gpu,30,16.00,32.00,533.33,1066.67,NaN,https://elinux.org/Jetson_AGX_Xavier
3,AMD RX480,gpu,150,5.80,5.80,38.67,38.67,8.0,techpowerup.com
4,GTX 1080 Ti,gpu,250,11.34,0.17,45.36,0.68,11.0,techpowerup.com


In [15]:
gpus_dataset = gpus_dataset[["name","type","tdp_watts"]]
gpus_dataset.head()

,name,type,tdp_watts
0,A100 PCIe 40/80GB,gpu,250
1,A100 SXM4 80 GB,gpu,400
2,AGX Xavier,gpu,30
3,AMD RX480,gpu,150
4,GTX 1080 Ti,gpu,250


In [20]:
gamma_dataset = pd.read_csv("gamma.csv")
gamma_datasets = gamma_dataset.drop('Country code', axis=1)

In [21]:
idx = gamma_datasets.groupby('Country or region')['Year'].idxmax()
gamma_data = gamma_datasets.loc[idx]
gamma_data = gamma_data.rename(columns={'Country or region': 'Regions', 'Emissions intensity (gCO2 per kWh)': 'Gamma'})
gamma_data.head()

,Regions,Year,Gamma
20,Afghanistan,2020,166.667
42,Africa,2021,489.260
63,Albania,2020,24.482
84,Algeria,2020,485.450
105,American Samoa,2020,687.500


In [22]:
pue_dataset = pd.read_csv("pue.csv")
pue_dataset = pue_dataset.rename(columns={'Cloud Provider': 'Providers'})
pue_dataset.head()

,Providers,PUE
0,Google Cloud,1.10
1,Microsoft Azure,1.18
2,Alibaba Cloud,1.25
3,Amazon Web Services,1.15
4,Oracle Cloud Infrastructure,1.31


In [23]:
from pandasql import sqldf

In [24]:
def data_parse(user_input,dataframe):
    query = ""
    if dataframe == "gpus_dataset":
        query = "SELECT G.tdp_watts FROM gpus_dataset AS G WHERE G.name LIKE '{}'".format(user_input)
    elif dataframe == "gamma_data":
        query = "SELECT G.Gamma FROM gamma_data AS G WHERE G.Regions LIKE '{}'".format(user_input)
    elif dataframe == "pue_dataset":
        query = "SELECT G.PUE FROM pue_dataset AS G WHERE G.Providers LIKE '{}'".format(user_input)
    result = sqldf(query, env=None)
    resultant = float(result.iloc[0][0])
    return resultant

In [30]:
result = data_parse("AGX Xavier","gpus_dataset")
print(type(result))

<class 'float'>


In [26]:
result2 = data_parse("Africa","gamma_data")
print(result2)

489.26


In [27]:
result2 = data_parse("Google Cloud","pue_dataset")
print(result2)

1.1


In [39]:
def calculate_carbon_emissions(region, providers,grid_config, hours_trained, energy, num_processors, avg_power_per_processor,hardware_type):
    """
    Calculate carbon emissions using the formula: CF = γ · PUE · (E_CPU + E_GPU + E_RAM)
    energy : kwh
    gamma : kg/kwh
    CF: kg
    Args:
        PUE: Power Usage Effectiveness
        E_CPU: Energy consumption of CPU (optional)
        E_GPU: Energy consumption of GPU (optional)
        E_RAM: Energy consumption of RAM (optional)
        gamma: Gamma coefficient
        hours_trained: Hours of training
        num_processors: Number of processors
        avg_power_per_processor: Average power per processor per hour
    Returns:
        The calculated carbon emissions in kg.
    """
    total_energy = 0
    
    if energy:
        total_energy = energy
        
    else:
        if data_parse(hardware_type,"gpus_dataset"):
            total_energy = hours_trained * data_parse(hardware_type,"gpus_dataset")
        else:
            if num_processors and avg_power_per_processor:
                total_energy = hours_trained * num_processors * avg_power_per_processor
                
        total_energy = float(total_energy/1000) # KWh   
        print(data_parse(hardware_type,"gpus_dataset"))
        
    if data_parse(region,"gamma_data"):
        gamma = data_parse(region,"gamma_data")
    else:
        gamma = 378.625
    
    gamma = float(gamma/1000) # kg/Kwh
        
    if data_parse(providers,"pue_dataset"):   # no units
        PUE = data_parse(providers,"pue_dataset")
    else:
        PUE = 1
    
    carbon_emissions = gamma * PUE * (total_energy)
    if grid_config:
        return carbon_emissions*grid_config
    else:
        return carbon_emissions


In [41]:
result = calculate_carbon_emissions("United States of America","Google Cloud",1,648,None,None,None,"TPUv3 Chip")
print(result)

283.0
76.37714369999999
